In [ ]:
import pandas as pd
import cv2
import numpy as np
import os
from ast import literal_eval as make_tuple

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/mini_project

/content/drive/.shortcut-targets-by-id/1lL1dCMD754gcws6p-RygR7zvteZO4gk2/mini_project


In [ ]:
pose_csv_path = "./dataset/pose_bb.csv"
wpn_csv_path = "./dataset/wpn_bb.csv"
dataset_path = "./dataset/cleaned/"

In [ ]:
pose_df = pd.read_csv(pose_csv_path)
wpn_df = pd.read_csv(wpn_csv_path)

In [ ]:
combined_df = pose_df.merge(wpn_df, how='left', on='name', suffixes=('_pose', '_wpn'))

In [ ]:
combined_df

,Unnamed: 0_pose,label_pose,name,shape,points_pose,original,Unnamed: 0_wpn,label_wpn,points_wpn
0,0,0,A0194484R_20220903_carrying_11130.29746_30.png,"(640, 960, 3)","[187.0769500732422, 514.323486328125, 168.1775...","(400, 640, 3)",0.0,0.0,"[204.80332946777344, 22.519760131835938, 640.0..."
1,1,0,A0194554W_20220904_carrying_12360.67851_30.png,"(960, 960, 3)","[350.8435363769531, 838.0150756835938, 279.798...","(1574, 1586, 3)",0.0,0.0,"[628.934814453125, 1020.5405883789062, 1013.53..."
2,2,0,0218358_20220831_carrying_54250.27364_30.png,"(960, 704, 3)","[121.80764770507812, 354.1222839355469, 276.46...","(720, 486, 3)",4.0,0.0,"[138.4716796875, 329.34832763671875, 476.86386..."
3,3,0,A0204735J_20220828_carrying_34560.081662_100.png,"(576, 960, 3)","[466.5370788574219, 492.9637451171875, 52.4743...","(720, 1280, 3)",3.0,0.0,"[62.598609924316406, 209.83041381835938, 746.8..."
4,4,0,A0254392B_20220904_carrying_00010.20247_30.png,"(576, 960, 3)","[712.0015258789062, 831.7986450195312, 203.513...","(782, 1366, 3)",1.0,0.0,"[6.372992038726807, 290.9029846191406, 1020.80..."
...,...,...,...,...,...,...,...,...,...
1497,45,2,0201603_20220829_normal_39190.72515_30.png,"(960, 576, 3)","[137.33834838867188, 273.24041748046875, 451.5...","(1920, 1080, 3)",NaN,NaN,NaN
1498,46,2,0218358_20220831_normal_21450.40522_30.png,"(576, 960, 3)","[482.5580749511719, 720.0970458984375, 114.580...","(720, 1280, 3)",NaN,NaN,NaN
1499,47,2,A0192318A_20220903_normal_12360.00078967_30.png,"(768, 960, 3)","[380.9498291015625, 639.3873901367188, 192.971...","(480, 640, 3)",NaN,NaN,NaN
1500,48,2,A0219715B_20220903_normal_72550.65296_30.png,"(576, 960, 3)","[481.4891052246094, 481.4891052246094, 528.008...","(360, 640, 3)",NaN,NaN,NaN


In [ ]:
normal_df = combined_df[combined_df['label_pose']==2].copy() 

In [ ]:
carrying_df = combined_df[combined_df['label_pose']==0].copy() 

In [ ]:
threat_df = combined_df[combined_df['label_pose']==1].copy() 

In [ ]:
padding = 70 # number of padding pixels

In [ ]:
def make_cropped_images(data_dir, bb_df, is_normal): # is_normal true when using df for normal -> no wpn
    if not os.path.isdir("./cropped"):
        os.mkdir("./cropped/")
    print("Could not crop images at index: ")
    for index, row in bb_df.iterrows():
        try: 
            name = row["name"]

            old_shape = row["original"]
            old_shape = make_tuple(old_shape)
            shape = row["shape"]
            shape = make_tuple(shape)

            path = data_dir + name
            img = cv2.imread(path)
            img = cv2.resize(img, (shape[1], shape[0]))

            pose_bb_pts = row["points_pose"]
            pose_bb_pts = make_tuple(pose_bb_pts)
            pose_bb_pts = tuple(int(val) for val in pose_bb_pts)

            if is_normal:       
                x_min = max(0, pose_bb_pts[0] - padding)
                y_min = max(0, pose_bb_pts[2], - padding)
                x_max = min(shape[1], pose_bb_pts[1] + padding)
                y_max = min(shape[0], pose_bb_pts[3] + padding)
            else: 
                wpn_bb_pts = row["points_wpn"]
                wpn_bb_pts = make_tuple(wpn_bb_pts)
                wpn_bb_pts = list(int(val) for val in pose_bb_pts) # scale because pose detection resizes ...
                wpn_bb_pts[0] = int(wpn_bb_pts[0]*shape[1]/old_shape[1]) # csvs bounding boxes are in diff format ...
                wpn_bb_pts[1] = int(wpn_bb_pts[1]*shape[0]/old_shape[0])
                wpn_bb_pts[2] = int(wpn_bb_pts[2]*shape[1]/old_shape[1])
                wpn_bb_pts[3] = int(wpn_bb_pts[3]*shape[0]/old_shape[0])
                x_min = max(0, min(pose_bb_pts[0], wpn_bb_pts[0]) - padding)
                y_min = max(0, min(pose_bb_pts[2], wpn_bb_pts[1]) - padding)
                x_max = min(shape[1], max(pose_bb_pts[1], wpn_bb_pts[2]) + padding)
                y_max = min(shape[0], max(pose_bb_pts[3], wpn_bb_pts[3]) + padding + 50) # more padding for head
            # some bug here: some images min, max are wrong. maybe csv issue. 
            cropped_img = img[y_min:y_max, x_min:x_max] 
            cv2.imwrite("cropped/" + name, cropped_img)
        except:
            print(index)
            cv2.imwrite("cropped/" + name, img)

In [ ]:
make_cropped_images(dataset_path + "normal/", normal_df, True)

Could not crop images at index: 
1039
1043
1069
1097
1114
1177
1249
1312
1313
1314
1315
1316
1326
1330


In [ ]:
make_cropped_images(dataset_path + "carrying/", carrying_df, False)

270
342


In [ ]:
make_cropped_images(dataset_path + "threat/", threat_df, False)

509
573
586
649
707
720
784
807
869
872
